In [ ]:
# For Google Colab Use 
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# %cd '/content/drive/MyDrive/Colab Notebooks/DeepFake Detector'
%cd '..'

In [ ]:
!pip install face_recognition

In [ ]:
import os
from tqdm import tqdm
from PIL import Image
from heapq import nsmallest
from math import ceil
from time import time
from random import randint, seed
from pipeline import *
seed(0)

In [ ]:
root_datasets_path = './Celeb-DF-v2'

In [ ]:
real_video_dir_path = f'{root_datasets_path}/Celeb-videos/Celeb-real'
fake_video_dir_path = f'{root_datasets_path}/Celeb-videos/Celeb-synthesis'

In [ ]:
# Create test set 5 male actors and females actors, respectively
test_males = [0,1,2,3,4]
test_females = [7,8,10,11,13]
test_actor_ids = ['id' + str(num) for num in test_males+test_females]

In [ ]:
video_dir_path = fake_video_dir_path
class_name = video_dir_path.split('-')[-1]

In [ ]:
for video_name in tqdm(os.listdir(video_dir_path)[10:]):
    video = os.path.join(video_dir_path, video_name)
    video_name = video_name.replace('.mp4', '')
    frames = get_every_frame(video, interval=1)

    face_coordinates = get_face_locations(frames, GPU=False, batch_size=96)
    centroid = get_centroid(face_coordinates)
    dist_from_centroid = get_distance_from_centroid(centroid, face_coordinates)

    movement_thershold = sorted(set([d for d in dist_from_centroid if d is not None]))[-2]
    zoomed_stable_faces = get_stable_faces(movement_thershold,
                                           dist_from_centroid,
                                           face_coordinates,
                                           frames,
                                           zoomed=True)
    # Create imgs for train-val set
    if not any([1 if actor_id in video_name else 0 for actor_id in test_actor_ids]):
        for i, sub_avg in enumerate(sub_average(frames, face_coordinates, interval=8)):
            Image.fromarray(sub_avg, mode='RGB').save(
                f'{root_datasets_path}/Celeb-avg/Celeb-{class_name}/{video_name}_{i}.png','PNG')

        for i, sub_diff in enumerate(sub_difference(frames, face_coordinates, interval=8)):
            Image.fromarray(sub_diff, mode='RGB').save(
                f'{root_datasets_path}/Celeb-diff/Celeb-{class_name}/{video_name}_{i}.png','PNG')

        for i, frame in enumerate(zoomed_stable_faces):
            Image.fromarray(frame, mode='RGB').save(
                f'{root_datasets_path}/Celeb-frames/Celeb-{class_name}/{video_name}_{i}.png','PNG')
    
    
    # Create imgs for test set     
    else:
        for i, sub_avg in enumerate(sub_average(frames, face_coordinates, interval=8)):
            Image.fromarray(sub_avg, mode='RGB').save(
                f'{root_datasets_path}/Celeb-avg-test/Celeb-{class_name}/{video_name}_{i}.png','PNG')

        for i, sub_diff in enumerate(sub_difference(frames, face_coordinates, interval=8)):
            Image.fromarray(sub_diff, mode='RGB').save(
                f'{root_datasets_path}/Celeb-diff-test/Celeb-{class_name}/{video_name}_{i}.png','PNG')

        for i, frame in enumerate(zoomed_stable_faces):
            Image.fromarray(frame, mode='RGB').save(
                f'{root_datasets_path}/Celeb-frames-test/Celeb-{class_name}/{video_name}_{i}.png','PNG')